In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import time

In [ ]:
class AKPD(object):

    def __init__(self, aws_credentials):
        self.client = boto3.client(
            "sagemaker-runtime", 
            region_name="eu-west-1", 
            aws_access_key_id=aws_credentials['aws_access_key_id'], 
            aws_secret_access_key=aws_credentials['aws_secret_access_key']
        
        )

    def predict_keypoints(self, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, camera_metadata):
        body = [{
            'leftCropUrl': left_crop_url,
            'rightCropUrl': right_crop_url,
            'leftCropMetadata': left_crop_metadata,
            'rightCropMetadata': right_crop_metadata,
            'cameraMetadata': camera_metadata,
            'id': 1
        }]

        body_str = json.dumps(body).replace("'", '"')

        resp = self.client.invoke_endpoint(EndpointName='auto-keypoints', ContentType='application/json', Body=body_str)
        akpd_keypoints_str = resp['Body'].read()
        akpd_keypoints = json.loads(akpd_keypoints_str.decode("utf-8"))
        return akpd_keypoints

<h1> Generate first DF </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM
    prod.crop_annotation cas
    INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
    WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
    AND cas.annotation is not null
    AND cas.pen_id=61
    AND cas.group_id='staging-b2-orig-61'
    AND cas.captured_at between '2019-09-13' and '2019-09-21';
"""

df = rds_access_utils.extract_from_database(query)
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

to_tensor_transform = ToTensor()

# initialize data transforms so that we can run inference with biomass neural network
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')
akpd_scorer_network = load_model('/root/data/alok/biomass_estimation/playground/akpd_scorer_model_TF.h5') # make this better

<h1> Function to generate weight prediction and confidence score </h1>

In [ ]:
def generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, akpd_keypoints, cm):
    
    # run AKPD scoring network
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    akpd_score = generate_confidence_score(input_sample, akpd_scorer_network)

    # run biomass estimation
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    akpd_weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    
    return akpd_score, akpd_weight_prediction


In [ ]:
weight_score_dict = {}

args = []
count = 0
for idx, row in df.iterrows():
    left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
    cm = row.camera_metadata
    akpd_keypoints = row.annotation
    row_id = idx
    akpd_score, akpd_weight_prediction = generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, akpd_keypoints, cm)
    weight_score_dict[row_id] = {
        'akpd_score': akpd_score,
        'akpd_weight_prediction': akpd_weight_prediction,
    }
    
    if count % 100 == 0:
        print(count)
    count += 1

In [ ]:
df['akpd_weight'], df['akpd_score'] = np.nan, np.nan
for idx, row in df.iterrows():
    if idx in weight_score_dict.keys():
        df.at[idx, 'akpd_weight'] = weight_score_dict[idx]['akpd_weight_prediction']
        df.at[idx, 'akpd_score'] = weight_score_dict[idx]['akpd_score']

df.index = pd.to_datetime(df.captured_at)

In [ ]:
df[df.akpd_score > 0.9].akpd_weight.mean()

<h1> Generate Second DF </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
query = """
    SELECT * FROM
    prod.crop_annotation cas
    INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
    WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
    AND cas.annotation is not null
    AND cas.pen_id=61
    AND cas.group_id='staging-61'
    AND cas.captured_at between '2019-09-13' and '2019-09-21';
"""

df2 = rds_access_utils.extract_from_database(query)

In [ ]:
weight_score_dict = {}

args = []
count = 0
for idx, row in df2.iterrows():
    left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
    cm = row.camera_metadata
    akpd_keypoints = row.annotation
    row_id = idx
    akpd_score, akpd_weight_prediction = generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, akpd_keypoints, cm)
    weight_score_dict[row_id] = {
        'akpd_score': akpd_score,
        'akpd_weight_prediction': akpd_weight_prediction,
    }
    
    if count % 100 == 0:
        print(count)
    count += 1

In [ ]:
df2.captured_at

In [ ]:
df2[df2.akpd_score > 0.9].akpd_weight.mean()

In [ ]:
(3045-3070)/3070

In [ ]:
df2['akpd_weight'], df2['akpd_score'] = np.nan, np.nan
for idx, row in df2.iterrows():
    if idx in weight_score_dict.keys():
        df2.at[idx, 'akpd_weight'] = weight_score_dict[idx]['akpd_weight_prediction']
        df2.at[idx, 'akpd_score'] = weight_score_dict[idx]['akpd_score']

df2.index = pd.to_datetime(df2.captured_at)

In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.annotation and 'rightCrop' in row.annotation:
        return pixel2world(row.annotation['leftCrop'], row.annotation['rightCrop'], row.camera_metadata)
    else:
        return None
df['world_keypoints'] = df.apply(
    lambda x: get_world_keypoints(x), axis=1
)


df2['world_keypoints'] = df2.apply(
    lambda x: get_world_keypoints(x), axis=1
)

df['dist'] = df.world_keypoints.apply(lambda x: euclidean_distance(x['ANAL_FIN'], x['EYE']))
df2['dist'] = df2.world_keypoints.apply(lambda x: euclidean_distance(x['ANAL_FIN'], x['EYE']))

In [ ]:
weights, keypoints, weights2, keypoints2 = [], [], [], []
dists, dists2 = [], []
for idx, row in df.iterrows():
    url = row.left_crop_url
    f_name = os.path.basename(url)
    search_url = url.replace('s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production', 
                             'aquabyte-crops.s3.eu-west-1.amazonaws.com/environment=staging')
    if df2.left_crop_url.str.contains(f_name).any() and row.akpd_score > 0.9:
        row2 = df2[df2.left_crop_url.str.contains(f_name)]
        score2 = row2.akpd_score.iloc[0]
        if score2 > 0.9:
            weight2 = row2.akpd_weight.iloc[0]
            weights.append(row.akpd_weight)
            weights2.append(weight2)
            keypoints.append(row.annotation)
            keypoints2.append(row2.annotation.iloc[0])
            dists.append(row.dist)
            dists2.append(row2.dist.iloc[0])
            

In [ ]:
np.mean(np.array(dists) - np.array(dists2))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(np.array(dists) - np.array(dists2))
plt.grid()
plt.show()

In [ ]:
[x for x in np.array(dists) - np.array(dists2)][17]

In [ ]:
x1 = df[(df.akpd_weight > 2302.56) & (df.akpd_weight < 2302.57)]
x2 = df2[(df2.akpd_weight > 2683.95) & (df2.akpd_weight < 2683.96)]

In [ ]:
x1.right_crop_url.iloc[0]

In [ ]:
# EYE TRIANGULATION
kps_left = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x1.annotation.iloc[0]['leftCrop']}
kps_right = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x1.annotation.iloc[0]['rightCrop']}
bp = 'EYE'

x_left = kps_left[bp][0]
y_left = kps_left[bp][1]
x_right = kps_right[bp][0]
y_right = kps_right[bp][1]
x_left_frame = x_left + x1.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x1.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x1.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x1.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x1.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_a = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_a)

# ANAL FIN TRIANGULATION

bp = 'ANAL_FIN'

x_left = kps_left[bp][0]
y_left = kps_left[bp][1]
x_right = kps_right[bp][0]
y_right = kps_right[bp][1]
x_left_frame = x_left + x1.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x1.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x1.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x1.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x1.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_b = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_b)

print(euclidean_distance(world_coordinates_a, world_coordinates_b))

In [ ]:
# EYE TRIANGULATION
kps_left = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x2.annotation.iloc[0]['leftCrop']}
kps_right = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x2.annotation.iloc[0]['rightCrop']}
bp = 'EYE'

x_left = kps_left[bp][0]
y_left = kps_left[bp][1]
x_right = kps_right[bp][0]
y_right = kps_right[bp][1]
x_left_frame = x_left + x2.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x2.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x2.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x2.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x2.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_a = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_a)

# ANAL FIN TRIANGULATION

bp = 'ANAL_FIN'

x_left = kps_left[bp][0]
y_left = kps_left[bp][1]
x_right = kps_right[bp][0]
y_right = kps_right[bp][1]
x_left_frame = x_left + x2.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x2.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x2.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x2.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x2.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_b = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_b)

print(euclidean_distance(world_coordinates_a, world_coordinates_b))

In [ ]:
x2.right_crop_url.iloc[0]

In [ ]:
# EYE TRIANGULATION
kps_left = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x1.annotation.iloc[0]['leftCrop']}
kps_right = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x1.annotation.iloc[0]['rightCrop']}
bp = 'EYE'

x_left = 1707
y_left = 190
x_right = 1710
y_right = 196
x_left_frame = x_left + x1.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x1.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x1.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x1.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
print(disp)
cm = x1.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_a = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_a)

# ANAL FIN TRIANGULATION

bp = 'ANAL_FIN'

x_left = 677
y_left = 494
x_right = 669
y_right = 502
x_left_frame = x_left + x1.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x1.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x1.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x1.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x1.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_b = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_b)

print(euclidean_distance(world_coordinates_a, world_coordinates_b))

In [ ]:
# EYE TRIANGULATION
kps_left = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x2.annotation.iloc[0]['leftCrop']}
kps_right = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in x2.annotation.iloc[0]['rightCrop']}
bp = 'EYE'

x_left = 1691
y_left = 192
x_right = 1697
y_right = 194
x_left_frame = x_left + x2.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x2.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x2.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x2.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
print(disp)
cm = x2.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_a = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_a)

# ANAL FIN TRIANGULATION

bp = 'ANAL_FIN'

x_left = 674
y_left = 490
x_right = 664
y_right = 499
x_left_frame = x_left + x2.left_crop_metadata.iloc[0]['x_coord']
y_left_frame = y_left + x2.left_crop_metadata.iloc[0]['y_coord']
x_right_frame = x_right + x2.right_crop_metadata.iloc[0]['x_coord']
y_right_frame = y_right + x2.right_crop_metadata.iloc[0]['y_coord']
disp = abs(x_right_frame - x_left_frame)
cm = x2.camera_metadata.iloc[0]
depth = depth_from_disp(disp, cm)
world_coordinates_b = convert_to_world_point(x_left_frame, y_left_frame, depth, cm)
print(world_coordinates_b)

print(euclidean_distance(world_coordinates_a, world_coordinates_b))

In [ ]:
body_part = 'UPPER_LIP'
d_diffs = []
for kps, kps2 in zip(keypoints, keypoints2):
    left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in kps['leftCrop']}
    right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in kps['rightCrop']}
    left_kps2 = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in kps2['leftCrop']}
    right_kps2 = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in kps2['rightCrop']}
    d = left_kps[body_part][0] - right_kps[body_part][0]
    d2 = left_kps2[body_part][0] - right_kps2[body_part][0]
    d_diffs.append(d - d2)